# dealing with text

In [3]:
import pandas as pd
import math
import re

In [4]:
news = pd.read_csv('news_mod_1.csv',index_col="ID")
news['title'] = news['title'].apply(lambda title:title.split(','))
news['uri'] = news['uri'].apply(lambda uri:uri.split(',') if type(uri) is str else [])
news['uri_features'] = news['uri_features'].apply(lambda f:f if type(f) is str else "")

In [5]:
news

,CATEGORY,PUBLISHER,genre,title,uri,story,uri_features,month,date,day,time,ts
ID,,,,,,,,,,,,
13,b,Reuters,article,"[Europe, reaches, crunch, point, on, banking, ...","[eu, banks, resoluton]",0,INL6N0M71NJ,3,10,Mon,0,8640.0
14,b,Reuters,article,"[ECB, FOCUS-Stronger, euro, drowns, out, ECB's...","[ecb, policy]",0,INL6N0M71K3,3,10,Mon,0,8640.0
20,b,Businessweek,news,"[Euro, Anxieties, Wane, as, Bunds, Top, Treasu...","[german, bonds, beat, treasuries, with, yield,...",0,,3,10,Mon,0,8640.0
21,b,Businessweek,news,"[Noyer, Says, Strong, Euro, Creates, Unwarrant...","[noyer, says, stronger, euro, creates, unwarra...",0,,3,10,Mon,0,8640.0
30,b,Reuters,article,"[REFILE-Bad, loan, triggers, key, feature, in,...","[eurozone, bank, tests]",0,INL6N0M71F0,3,10,Mon,0,8640.0
...,...,...,...,...,...,...,...,...,...,...,...,...
422819,m,Huffington Post,hp,"[UN, Ebola, Could, Eventually, Infect, 20000, ...","[ebola, 20000, people]",4577,,8,28,Thu,0,254880.0
422820,m,Reuters,article,"[Ebola, toll, tops, 1550, continues, to, accel...","[health, ebola, toll]",4577,INL5N0QY2GR,8,28,Thu,0,254880.0
422821,m,Reuters,article,"[UPDATE, 1-Ebola, toll, tops, 1550, outbreak, ...","[health, ebola, toll]",4577,INL5N0QY2J7,8,28,Thu,0,254880.0


## 表記ゆれ・語幹を抽出

In [32]:
news['uri_mod'] = news['uri']
news['title_mod'] = news['title']

- 一度辞書を作る

In [33]:
def create_dic(series,n):
    dic = {}
    for li in series:
        for elm in li:
            dic.setdefault(elm,0)
            dic[elm] += 1
    dic = {k:v for k,v in dic.items() if v>n}
    return dic

In [34]:
print(*["{}回出現:{}種".format(i,len(create_dic(news['uri_mod']+news['title_mod'],i))) for i in range(10)],sep="\n")

0回出現:31917種
1回出現:17743種
2回出現:12274種
3回出現:9660種
4回出現:7892種
5回出現:6763種
6回出現:5881種
7回出現:5192種
8回出現:4658種
9回出現:4219種


- 末尾のsを取り除く

In [35]:
remove_s = lambda li,keys: [e.rstrip('s') if len(e)>1 and e[-1]=="s" and e.rstrip('s') in keys else e for e in li]
remove_S = lambda li,keys: [e.rstrip('S') if len(e)>1 and e[-1]=="S" and e.rstrip('S') in keys else e for e in li]

In [36]:
keys = create_dic(news['uri_mod']+news['title_mod'],0).keys()
news['uri_mod'] = news['uri_mod'].apply(lambda li:remove_S(remove_s(li, keys),keys))
news['title_mod'] = news['title_mod'].apply(lambda li:remove_S(remove_s(li, keys),keys))

In [38]:
print(*["{}回出現:{}種".format(i,len(create_dic(news['uri_mod']+news['title_mod'],i))) for i in range(10)],sep="\n")

0回出現:28828種
1回出現:16124種
2回出現:11254種
3回出現:8973種
4回出現:7405種
5回出現:6405種
6回出現:5614種
7回出現:4969種
8回出現:4461種
9回出現:4074種


In [43]:
news[['title','uri','title_mod','uri_mod']]

,title,uri,title_mod,uri_mod
ID,,,,
13,"[Europe, reaches, crunch, point, on, banking, ...","[eu, banks, resoluton]","[europe, reaches, crunch, point, on, banking, ...","[eu, bank, resoluton]"
14,"[ECB, FOCUS-Stronger, euro, drowns, out, ECB's...","[ecb, policy]","[ECB, FOCUS-Stronger, euro, drowns, out, ECB's...","[ecb, policy]"
20,"[Euro, Anxieties, Wane, as, Bunds, Top, Treasu...","[german, bonds, beat, treasuries, with, yield,...","[euro, anxieties, Wane, a, Bunds, top, treasur...","[german, bond, beat, treasuries, with, yield, ..."
21,"[Noyer, Says, Strong, Euro, Creates, Unwarrant...","[noyer, says, stronger, euro, creates, unwarra...","[noyer, say, strong, euro, create, unwarranted...","[noyer, say, stronger, euro, create, unwarrant..."
30,"[REFILE-Bad, loan, triggers, key, feature, in,...","[eurozone, bank, tests]","[REFILE-Bad, loan, trigger, key, feature, in, ...","[eurozone, bank, test]"
...,...,...,...,...
422819,"[UN, Ebola, Could, Eventually, Infect, 20000, ...","[ebola, 20000, people]","[UN, ebola, could, eventually, infect, 20000, ...","[ebola, 20000, people]"
422820,"[Ebola, toll, tops, 1550, continues, to, accel...","[health, ebola, toll]","[ebola, toll, top, 1550, continue, to, acceler...","[health, ebola, toll]"
422821,"[UPDATE, 1-Ebola, toll, tops, 1550, outbreak, ...","[health, ebola, toll]","[UPDATE, 1-Ebola, toll, top, 1550, outbreak, a...","[health, ebola, toll]"


- 最初の文字だけ大文字のパターンは小文字に戻す

In [39]:
recover = lambda li,keys: [word.lower() if len(word)>1 and 'A'<= word[0] <='Z' and sum(['A'<=c<='Z' for c in word[1:]]) == 0 and word.lower() in keys else word for word in li]

In [40]:
keys = create_dic(news['uri_mod']+news['title_mod'],0).keys()
news['uri_mod'] = news['uri_mod'].apply(lambda li:recover(li, keys))
news['title_mod'] = news['title_mod'].apply(lambda li:recover(li, keys))

In [41]:
print(*["{}回出現:{}種".format(i,len(create_dic(news['uri_mod']+news['title_mod'],i))) for i in range(10)],sep="\n")

0回出現:21292種
1回出現:12824種
2回出現:8751種
3回出現:7200種
4回出現:6057種
5回出現:5297種
6回出現:4676種
7回出現:4241種
8回出現:3839種
9回出現:3527種


- 全部大文字のパターンは全部小文字に戻す

In [15]:
recover_2 = lambda li,keys: [word.lower() if sum([c<'A' or 'Z'<c for c in word]) == 0 and word.lower() in keys else word for word in li]

In [16]:
keys = create_dic(news['uri_mod']+news['title_mod'],0).keys()
news['uri_mod'] = news['uri_mod'].apply(lambda li:recover_2(li, keys))
news['title_mod'] = news['title_mod'].apply(lambda li:recover_2(li, keys))

In [17]:
print(*["{}回出現:{}種".format(i,len(create_dic(news['uri_mod']+news['title_mod'],i))) for i in range(10)],sep="\n")

0回出現:22581種
1回出現:13843種
2回出現:9463種
3回出現:7747種
4回出現:6504種
5回出現:5682種
6回出現:4997種
7回出現:4544種
8回出現:4130種
9回出現:3796種


- (title, uri)についてPPMIを計算

In [21]:
def gen_ppmi(series, keys=None):
    dic = {}
    for line in series:
        for w1 in line:
            if keys and w1 not in keys: continue
            dic.setdefault(w1,{})
            for w2 in line:
                if keys and w2 not in keys: continue
                if w1==w2: continue
                dic[w1].setdefault(w2,0)
                dic[w1][w2] += 1
    counts = {w:sum(v.values()) for w,v in dic.items()}
    s = sum(counts.values())
    return lambda w1,w2,count=False: 0 if w1 not in dic.keys() or w2 not in dic[w1].keys() else dic[w1][w2] if count else math.log2(dic[w1][w2]*s/counts[w1]/counts[w2])

In [22]:
ppmi = gen_ppmi(news['uri_mod']+news['title_mod'],create_dic(news['uri_mod']+news['title_mod'],2).keys())

In [23]:
def ppmis(dic,n=None):
    ppmi_dic = {}
    for w1 in dic:
        for w2 in dic:
            if (w2,w1) in ppmi_dic.keys(): continue
            if (o := ppmi(w1,w2)) > 0:
                ppmi_dic[(w1,w2)] = o
    if n is None: return ppmi_dic
    return {k:v for k,v in sorted(ppmi_dic.items(), key=lambda x:x[1], reverse=True)[:n]}

In [24]:
pp = ppmis(create_dic(news['uri_mod']+news['title_mod'],2).keys())

In [25]:
sorted(pp.items(), key=lambda x:x[1], reverse=True)[:50]

[(('decarbonization', 'pathway'), 15.234705438418128),
 (('morningcall', 'Call-Global'), 14.7316279044262),
 (('ifr', 'regional'), 14.527487187645974),
 (('biogen', 'idec'), 14.509565279648712),
 (('nate', 'berku'), 14.357562186203662),
 (('jordana', 'brewster'), 14.296532752806103),
 (('barkhad', 'abdi'), 14.296532752806103),
 (('biogen', 'hemophilia'), 14.242084968783725),
 (('idec', 'hemophilia'), 14.242084968783725),
 (('subway', 'quizno'), 14.23798057045099),
 (('brominated', 'powerade'), 14.205559092758612),
 (('hercule', 'dwayne'), 14.090081875338676),
 (('silevo', 'SolarCity'), 14.068055569008678),
 (('loch', 'ne'), 14.053555999313563),
 (('cirru', 'wolfson'), 14.053555999313563),
 (('wally', 'pfister'), 14.01934028397565),
 (('arianna', 'huffington'), 13.994662310259994),
 (('sandisk', 'Fusion-io'), 13.994662310259994),
 (('vin', 'diesel'), 13.883630997871249),
 (('prdct', 'chem'), 13.772599685482506),
 (('cubist', 'anti-infective'), 13.72013226558837),
 (('dixon', 'carphone')

In [26]:
ppmi("US","us")

0

## 連続語を捉える

### countinuous(語の前出現語の種類の数を調べる)

In [ ]:
def continuous(series,*,n=2):
    dic = {}
    for lis in series:
        for i in range(len(lis)-1):
            dic.setdefault(lis[i+1],set())
            dic[lis[i+1]].add(lis[i])
    return dic

In [ ]:
{k:len(v) for k,v in continuous(news['uri']).items()}

In [ ]:
def get_ngram(series,*,n,count_thr=None,rate_thr=None):
    dic = {}
    for lis in series:
        if len(lis)<n: continue
        for i in range(len(lis)-n+1):
            dic.setdefault(tuple(lis[i:i+n]),{'count':0})
            dic[tuple(lis[i:i+n])]['count'] += 1
    s = sum([v['count'] for v in dic.values()])
    dic = {k:{"count":v["count"],"rate":v['count']/s} for k,v in dic.items()}
    if count_thr: dic = {k:v for k,v in dic.items() if v['count']>count_thr}
    if rate_thr:  dic = {k:v for k,v in dic.items() if v['rate']>rate_thr}
    return dic

In [ ]:
def concat_words(li,ng):
    for i,w in enumerate(li):
        if i > len(li)-len(ng): break
        if sum([w1!=w2 for w1,w2 in zip(li[i:i+len(ng)],ng)])==0:
            li[i] = "_".join(li[i:i+len(ng)])
            for j in range(i+len(ng)-1,i,-1): li.pop(j)
    return li

In [ ]:
news['uri_cat'] = news['uri']

In [ ]:
for ng in get_ngram(news['uri_cat'],n=2,rate_thr=0.001).keys():
    news['uri_cat'] = news['uri_cat'].apply(lambda u:concat_words(u,ng))

In [ ]:
for ng in get_ngram(news['uri_cat'],n=2,rate_thr=0.001).keys():
    news['uri_cat'] = news['uri_cat'].apply(lambda u:concat_words(u,ng))

In [ ]:
get_ngram(news['uri_cat'],n=2,rate_thr=0.0001)